# Pseudo-label generator showcase

In [ ]:
from IPython.display import display, HTML
import os

os.environ["UNICORE_DATASETS"] = os.path.expanduser("~/datasets")

import unipercept as up  # isort: skip

pseudogen = up.data.sets.PseudoGenerator()
image = up.data.io.read_image("//datasets/kitti-step/testing/image_02/0000/000000.png")
info = up.data.sets.kitti_step.KITTISTEPDataset.read_info()

up.render.draw_image(image)

## Depth pseudo-label generation

### External model

We can generate depth labels using an external model.

In [ ]:
output_depth = pseudogen.estimate_depth(image)

up.render.draw_image_depth(output_depth, info=info)

maximum_depth = output_depth.max()
minimum_depth = output_depth.min()

display(
    HTML(f"<table><tr><td>{minimum_depth}</td><td>{maximum_depth}</td></tr></table>")
)

## Panoptic (proof of concept)

In [ ]:
import requests
import torch
from PIL import Image
from transformers import AutoImageProcessor, Mask2FormerForUniversalSegmentation


# load Mask2Former fine-tuned on Cityscapes panoptic segmentation
processor = AutoImageProcessor.from_pretrained(
    "facebook/mask2former-swin-large-cityscapes-panoptic"
)
model = Mask2FormerForUniversalSegmentation.from_pretrained(
    "facebook/mask2former-swin-large-cityscapes-panoptic"
)

image = Image.open(
    file_io.get_local_path("//datasets/kitti-step/testing/image_02/0000/000000.png")
)
inputs = processor(images=image, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

# model predicts class_queries_logits of shape `(batch_size, num_queries)`
# and masks_queries_logits of shape `(batch_size, num_queries, height, width)`
class_queries_logits = outputs.class_queries_logits
masks_queries_logits = outputs.masks_queries_logits

# you can pass them to processor for postprocessing
result = processor.post_process_panoptic_segmentation(
    outputs, target_sizes=[image.size[::-1]]
)[0]
# we refer to the demo notebooks for visualization (see "Resources" section in the Mask2Former docs)
predicted_panoptic_map = result["segmentation"]